In [3]:
import pandas as pd
import numpy as np
import os
import re


In [12]:
## this takes in a file & outputs buildings with all their labs

file_path = "All Fume Hoods List 11-2-22 - Sheet1.csv"
df = pd.read_csv(file_path)

# Get all unique building names
unique_buildings = df["oli"].dropna().unique()

output_dir = "./Rooms"
os.makedirs(output_dir, exist_ok=True)

saved_files = []

for building_name in unique_buildings:
    start_index = df[df["oli"] == building_name].index.min()

    next_building_index = df[df.index > start_index][df["oli"].notna()].index.min()

    if pd.isna(next_building_index):  # If this is the last building in the dataset
        building_df = df.loc[start_index:]
    else:
        building_df = df.loc[start_index:next_building_index - 1]

    filename = f"{output_dir}/{building_name.replace(' ', '_').replace('/', '_')}.csv"

    # Save extracted data to CSV
    building_df.to_csv(filename, index=False)
    saved_files.append(filename)

saved_files

/var/folders/dm/qcxk0by517nbn3hv449blw1c0000gn/T/ipykernel_12138/515675431.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  next_building_index = df[df.index > start_index][df["oli"].notna()].index.min()
/var/folders/dm/qcxk0by517nbn3hv449blw1c0000gn/T/ipykernel_12138/515675431.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  next_building_index = df[df.index > start_index][df["oli"].notna()].index.min()
/var/folders/dm/qcxk0by517nbn3hv449blw1c0000gn/T/ipykernel_12138/515675431.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  next_building_index = df[df.index > start_index][df["oli"].notna()].index.min()
/var/folders/dm/qcxk0by517nbn3hv449blw1c0000gn/T/ipykernel_12138/515675431.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  next_building_index = df[df.index > start_index][df["oli"].notna()].index.min()
/var/folders/dm/qcxk0by517nbn3hv449blw1c

['./Rooms/Ag_Waste_Management_Lb.csv',
 './Rooms/Agronomy_Leland_Laboratory.csv',
 './Rooms/Arthropod_Research_Facility.csv',
 './Rooms/Baker_Inst_Micro_Wing.csv',
 './Rooms/Baker_Institute_CRLDD.csv',
 './Rooms/Baker_Institute_Main_Building.csv',
 './Rooms/Baker_Institute_West_Wing.csv',
 './Rooms/Baker_Laboratory.csv',
 './Rooms/Bard_Hall.csv',
 './Rooms/Basic_Science_Building.csv',
 './Rooms/Biotechnology.csv',
 './Rooms/Bluegrass_L_Pesticide_Stg.csv',
 './Rooms/Boyce_Thompson_Institute.csv',
 './Rooms/Bradfield_Hall.csv',
 './Rooms/Brown_Rd_95_#6_Langmuir_Lab.csv',
 './Rooms/CALS_Surge_Facility.csv',
 './Rooms/Clark_Hall.csv',
 './Rooms/Clinical_Programs_K_Barn.csv',
 './Rooms/Clinical_Programs_Multipurpose.csv',
 './Rooms/Comstock_Hall.csv',
 './Rooms/Cornell_Health.csv',
 './Rooms/Dale_R_Corson_Hall.csv',
 './Rooms/Diagnostic_Lab_North.csv',
 './Rooms/Duffield_Hall.csv',
 './Rooms/Eco-Toxicology_Fish_Hatchery.csv',
 './Rooms/Emerson_Hall.csv',
 './Rooms/Environmental_Health_&_Saf

In [13]:
## read through the rooms on the building csv 
## make a template of how many fumehood they have by going through the 
# file_path = "All Fume Hoods List 11-2-22 - Sheet1.csv"

# this grabs data from rooms and put them into a csv in the processed_room folder with the correct format of: 
# building	| hood	| lab | floor | server | flow_sensor | 	sash_position_sensor | object_id | device_id

# Input and output directories
input_dir = "./Rooms"
output_dir = "./normalized_processed_rooms"
os.makedirs(output_dir, exist_ok=True)

# Process each building file
for file_name in os.listdir(input_dir):
    if not file_name.endswith(".csv"):
        continue

    file_path = os.path.join(input_dir, file_name)
    df = pd.read_csv(file_path)

    # Check for necessary columns
    if "Asset Description" not in df.columns or "PM Location" not in df.columns:
        continue

    df = df.dropna(subset=["Asset Description", "PM Location"])

    # Extract lab and floor
    df["lab"] = df["PM Location"].astype(str).str.replace("room", "", case=False, regex=True).str.strip()
    df["floor"] = df["lab"].str[0]

    # Assign hood numbers per lab
    df["hood"] = df.groupby("lab").cumcount() + 1

    # Standardize remaining columns
    df["building"] = file_name.replace(".csv", "").replace("_", " ")
    df["server"] = "biotech_main"
    df["flow_sensor"] = None
    df["sash_position_sensor"] = None
    df["object_id"] = None
    df["device_id"] = None

    # Reorder and select final columns
    formatted_df = df[[
        "building", "hood", "lab", "floor",
        "server", "flow_sensor", "sash_position_sensor", "object_id", "device_id"
    ]]

    # Save to output
    output_path = os.path.join(output_dir, file_name)
    formatted_df.to_csv(output_path, index=False)

### Hoodnaming

This notebook converts the point names downloaded from WebCTRL into a list of pseudo point names for later use

In [14]:
pointnames = pd.read_csv("./building_csv/biotech_all.csv", skiprows=22)
# pointnames = pd.read_csv("./building_csv/baker_all.csv")

# pointnames = pd.read_csv("./building_csv/kimball_all.csv")
pointnames = pointnames.rename(columns={"Object ID": "ObjectId", "Device ID": "DeviceId", "Object Name": "ObjectName"})
pointnames.head()


,Location,Control Program,Name,Type,ObjectId,DeviceId,ObjectName,Path
0,/Cornell University/Biotech Server/Biotech/Bio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"Biotech ACS-1, 2, and 3 Performance",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,AHU Exhaust Airflow,BAV,AV:16,DEV:1018006,ahu_exhaust_airflow_av_2,#biotech_acs-1_2_and_3_performance/ahu_exhaust...
3,NaN,NaN,AHU Outside Airflow,BAV,AV:17,DEV:1018006,ahu_outside_airflow_av_2,#biotech_acs-1_2_and_3_performance/ahu_outside...
4,NaN,NaN,AHU Supply Airflow,BAV,AV:18,DEV:1018006,ahu_supply_airflow_av_2,#biotech_acs-1_2_and_3_performance/ahu_supply_...


In [19]:
import pandas as pd
import re

# Load the normalized Biotechnology CSV
normalized_path = "./normalized_processed_rooms/Biotechnology.csv"
normalized_df = pd.read_csv(normalized_path)

# Load pointnames (control program data)
pointnames = pd.read_csv("./building_csv/biotech_all.csv", skiprows=22)

# === STEP 1: Fill lab labels from Control Program ===
pointnames["lab"] = pointnames["Control Program"].fillna(method="ffill")

# === STEP 2: Filter for flow sensor rows ===
flow_df = pointnames[
    pointnames["Name"].str.contains(r"(?i)hood flow|flow feedback", na=False)
]
flow_df = flow_df[flow_df["Type"].str.fullmatch(r"(?i)bai", na=False)]

# === STEP 3: Extract room number like '141', '203B' from lab label ===
def extract_room_number_flexible(label):
    if pd.isna(label):
        return None
    match = re.search(r'\b(?:Lab|Room)?\s*(\d+[A-Z]?)', label, re.IGNORECASE)
    return match.group(1) if match else None

flow_df["lab_extracted"] = flow_df["lab"].apply(extract_room_number_flexible)

# === STEP 4: Merge flow sensor paths into the normalized hood data ===
merged_df = pd.merge(
    normalized_df,
    flow_df[["lab_extracted", "Path"]],
    how="left",
    left_on=normalized_df["lab"].astype(str).str.upper(),
    right_on=flow_df["lab_extracted"].astype(str).str.upper()
)

# === STEP 5: Set flow_sensor column to the Path ===
merged_df["flow_sensor"] = merged_df["Path"]

# === STEP 6: Final cleanup ===
merged_df.drop(columns=["lab_extracted", "Path", "key_0"], errors="ignore", inplace=True)

# === STEP 7: Save back to same file ===
merged_df.to_csv("./normalized_processed_rooms/Biotechnology.csv", index=False)


/var/folders/dm/qcxk0by517nbn3hv449blw1c0000gn/T/ipykernel_12138/2924242340.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pointnames["lab"] = pointnames["Control Program"].fillna(method="ffill")


In [8]:
# building	| hood	| lab | floor | server | flow_sensor | 	sash_position_sensor | object_id | device_id
#  (obj id and device id of the sash)

def sash_pos_finder(df):
    # Filter for rows where "Name" is "Sash"
    df_filtered = df[df["Name"].str.contains(r"(?i)sash", na=False)].copy() 
    
    # filter for "BAI" in the type column 
    if "Type" in df_filtered.columns:
        df_filtered = df_filtered[df_filtered["Type"].str.fullmatch(r"(?i)(bai|asvi)", na=False)]

    # Save to CSV
    df_filtered.to_csv("sash.csv", index=False)
    
    # Print the filtered DataFrame
    print(df_filtered)

# Run the function using `pointnames`
sash_pos_finder(pointnames)


      Location Control Program                             Name Type ObjectId  \
2001       NaN             NaN    Fume Hood Sash Position Input  BAI     AI:3   
2192       NaN             NaN                    HOOD SASH POS  BAI     AI:4   
4792       NaN             NaN                    HOOD SASH POS  BAI     AI:4   
4839       NaN             NaN                    HOOD SASH POS  BAI     AI:4   
4914       NaN             NaN                    HOOD SASH POS  BAI     AI:4   
...        ...             ...                              ...  ...      ...   
22215      NaN             NaN                    HOOD SASH POS  BAI     AI:4   
22262      NaN             NaN                    HOOD SASH POS  BAI     AI:4   
22309      NaN             NaN                    HOOD SASH POS  BAI     AI:4   
22480      NaN             NaN  Fume Hood-1 Sash Position Input  BAI     AI:4   
22726      NaN             NaN  Fume Hood-1 Sash Position Input  BAI     AI:4   

          DeviceId       Ob

In [18]:
df = combined.copy()
floor_map = {"ground":"Ground", "1st":"First","2nd":"Second", "3rd":"Third","4th":"Fourth"}
df["FloorNum"] = df["Floor"].map(floor_map)
display(df)

,building,hood,Lab,Floor,Server,Flow Sensor,Sash,Occupancy,Internal Temp,External Temp,FloorNum
0,biotech,1,G54,G,biotech_main,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
1,biotech,1,141,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
2,biotech,1,143,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
3,biotech,1,147B,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
4,biotech,1,153,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
61,biotech,1,445B,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
62,biotech,1,453,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
63,biotech,1,453B,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN
64,biotech,1,457,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN


In [19]:
#Biotech.floor_num.lab_num.hood_num.{pointType}
#4 Point Types:[energy.occ,energy.unocc,sashOpenTime.occ, sashOpenTime.unocc]

def point_maker(lab,floor,hood):
    nstr = "Biotech." + str(floor) + "Floor" + ".Lab_" +  str(lab)   + ".Hood_" + str(hood) + "."
    nlist = [nstr+"energy.occ",nstr+"energy.unocc",nstr+"sashOpenTime.occ",nstr+"sashOpenTime.unocc"]
    return nlist



#display(df)

# test_case = point_maker(141,1,1)
# test_case

def make_all_points(df):
    energy_occ = []
    energy_unocc = []
    sashOpenTime_occ = []
    sashOpenTime_unocc = []
    for i in range(len(df["FloorNum"])):
        response = point_maker(df["Lab"][i],df["FloorNum"][i],1)
        energy_occ.append(response[0])
        energy_unocc.append(response[1])
        sashOpenTime_occ.append(response[2])
        sashOpenTime_unocc.append(response[3])
    df["Energy Occupied"] = energy_occ
    df["Energy Unoccupied"] = energy_unocc
    df["Sash Open Time Occupied"] = sashOpenTime_occ
    df["Sash Open Time Unoccupied"] = sashOpenTime_unocc
    return df

ndf = make_all_points(df)
display(ndf)







,building,hood,Lab,Floor,Server,Flow Sensor,Sash,Occupancy,Internal Temp,External Temp,FloorNum,Energy Occupied,Energy Unoccupied,Sash Open Time Occupied,Sash Open Time Unoccupied
0,biotech,1,G54,G,biotech_main,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_G54.Hood_1.energy.occ,Biotech.nanFloor.Lab_G54.Hood_1.energy.unocc,Biotech.nanFloor.Lab_G54.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_G54.Hood_1.sashOpenTime.u...
1,biotech,1,141,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_141.Hood_1.energy.occ,Biotech.nanFloor.Lab_141.Hood_1.energy.unocc,Biotech.nanFloor.Lab_141.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_141.Hood_1.sashOpenTime.u...
2,biotech,1,143,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_143.Hood_1.energy.occ,Biotech.nanFloor.Lab_143.Hood_1.energy.unocc,Biotech.nanFloor.Lab_143.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_143.Hood_1.sashOpenTime.u...
3,biotech,1,147B,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_147B.Hood_1.energy.occ,Biotech.nanFloor.Lab_147B.Hood_1.energy.unocc,Biotech.nanFloor.Lab_147B.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_147B.Hood_1.sashOpenTime....
4,biotech,1,153,1,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_153.Hood_1.energy.occ,Biotech.nanFloor.Lab_153.Hood_1.energy.unocc,Biotech.nanFloor.Lab_153.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_153.Hood_1.sashOpenTime.u...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,biotech,1,445B,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_445B.Hood_1.energy.occ,Biotech.nanFloor.Lab_445B.Hood_1.energy.unocc,Biotech.nanFloor.Lab_445B.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_445B.Hood_1.sashOpenTime....
62,biotech,1,453,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_453.Hood_1.energy.occ,Biotech.nanFloor.Lab_453.Hood_1.energy.unocc,Biotech.nanFloor.Lab_453.Hood_1.sashOpenTime.occ,Biotech.nanFloor.Lab_453.Hood_1.sashOpenTime.u...
63,biotech,1,453B,4,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,NaN,Biotech.nanFloor.Lab_453B.Hood_1.energy.occ,Biotech.nanFloor.Lab_453B.Hood_1.energy.unocc,Biotech.nanFloor.Lab_453B.Hood_1.sashOpenTime.occ,Biotech.nanFloo